# Building molecular structures with Libra (System class)

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)


2. [Manual building of the system (Molecular structure and topology)](#2.) 

  2.1. [Creating the System and adding ATOMs](#2.1.)
  
  2.2. [Linking and grouping atoms](#2.2.)
  
  2.3. [Functional group identification](#2.3.)
    

3. [Automatic building of a System object using the `LoadMolecule` module](#3.) 

  3.1. [Systems without rings](#3.1.)
  
  3.2. [Systems with rings](#3.2.)
  
  3.3. [Loading multiple molecules](#3.3.)
  
  3.4. [Constructing periodic Lattices](#3.4.)


4. [Manipulating System objects](#4.) 

  4.1. [Rotating fragments](#4.1.)
  
  4.2. [Translating fragments](#4.2.)
  

### A. Learning objectives

- to build molecular system from scratch, link, and group atoms
- to build molecular systems by loading files
- to learn some supported file formats
- to rotate and translate molecular fragments 


### B. Use cases

- rotational and translational scan of PES


### C. Functions

- `libra_py`  
  - `LoadMolecule`
    - [`Load_Molecule`](#Load_Molecule-1)
  - `LoadPT`
    - [`Load_PT`](#Load_PT-1)  
    
- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`System`](#System-1)
      - [`CREATE_ATOM`](#CREATE_ATOM-1) | [`CREATE_ATOM`](#CREATE_ATOM-2)
      - [`determine_functional_groups`](#determine_functional_groups-1)      
      - [`GROUP_ATOMS`](#GROUP_ATOMS-1)
      - [`init_box`](#init_box-1)
      - [`init_fragments`](#init_fragments-1)
      - [`LINK_ATOMS`](#LINK_ATOMS-1)
      - [`print_ent`](#print_ent-1) 
      - [`print_xyz`](#print_xyz-1)
      - [`ROTATE_FRAGMENT`](#ROTATE_FRAGMENT-1) | [`ROTATE_FRAGMENT`](#ROTATE_FRAGMENT-2)
      - [`show_atoms`](#show_atoms-1)
      - [`show_bonds`](#show_bonds-1)      
      - [`show_fragments`](#show_fragments-1)
      - [`show_molecules`](#show_molecules-1)
      - [`show_rings`](#show_rings-1)
      - [`TRANSLATE_FRAGMENT`](#TRANSLATE_FRAGMENT-1)
      
  - `libmol`
    - `Atom`
      - [`Atom`](#Atom-1)
      - [`save`](#save-1)
      - [`show_info`](#show_info-1)              
      

### D. Classes and class members

- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`System`](#System-1)
      - [`Atoms`](#Atoms-1)
      - [`Fragments`](#Fragments-1)
      - [`Number_of_atoms`](#Number_of_atoms-1)
      - [`Number_of_angles`](#Number_of_angles-1)
      - [`Number_of_bonds`](#Number_of_bonds-1)      
      - [`Number_of_dihedrals`](#Number_of_dihedrals-1)
      - [`Number_of_fragments`](#Number_of_fragments-1)
      - [`Number_of_impropers`](#Number_of_impropers-1)
      
  - `libmol`
    - `Atom`
      - [`Atom`](#Atom-1)
      - [`Atom_id`](#Atom_id-1)
      - [`Atom_RB`](#Atom_RB-1)
      - [`globAtom_Index`](#globAtom_Index-1)        
      
  - `libuniverse`
      - `Universe`
        - [`Universe`](#Universe-1)
    

## 1. General setups
<a name="setups"></a>[Back to TOC](#TOC)

Let's import all the needed libraries, including the `py3Dmol` library for visualizing the structures we generate.

In [1]:
import math
import liblibra_core
from liblibra_core import *
from libra_py import units
from libra_py import LoadPT, LoadMolecule

import os, sys
#stdout = sys.stdout

import py3Dmol
import matplotlib.pyplot as plt
#from matplotlib.pyplot import figure
%matplotlib inline

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::v

## 2. Manual building of the system (Molecular structure and topology)
<a name="2."></a>[Back to TOC](#TOC)

### 2.1. Creating the System and adding ATOMs
<a name="2.1."></a>[Back to TOC](#TOC)

Create the Universe object and populate it. The universe object contains information about periodic table (e.g. masses of atoms, their radii, "colors", etc.) and other  parameters. 

These properties are defined in the "elements.dat" file which we load
<a name="Load_PT-1"></a><a name="Universe-1"></a>

In [2]:
U = Universe()
verbose = 1
LoadPT.Load_PT(U, "elements.dat", verbose)

load element Xu
load element Xd
load element H
load element He
load element Li
load element Be
load element B
load element C
load element N
load element O
load element F
load element Ne
load element Na
load element Mg
load element Al
load element Si
load element P
load element S
load element Cl
load element Ar
load element K
load element Ca
load element Sc
load element Ti
load element V
load element Cr
load element Mn
load element Fe
load element Co
load element Ni
load element Cu
load element Zn
load element Ga
load element Ge
load element As
load element Se
load element Br
load element Kr
load element Rb
load element Sr
load element Y
load element Zr
load element Nb
load element Mo
load element Tc
load element Ru
load element Rh
load element Pd
load element Ag
load element Cd
load element In
load element Sn
load element Sb
load element Te
load element I
load element Xe
load element Cs
load element Ba
load element Pr
load element Nd
load element Sm
load element Eu
load element Gd
load

To start off, we demonstrate the `Atom` class available as one of the basic classes for the chemical system description.

The `Atom` class has a number of properties and member functions that are accessible via Python directly. 

Here, we demonstrate only some of them:
<a name="Atom_id-1"></a><a name="globAtom_Index-1"></a><a name="show_info-1"></a><a name="save-1"></a><a name="Atom-1"></a>

- `Atom_id` - the "ID" of the atom - could be any number;
- `globAtom_Index` - index of the Atom in the global array of atoms, starts with 0
- `show_info` - printing out the key properties to standard output
- `save` - this is a way to store properties of the atom in an XML file in an ordered way

In [3]:
uni1 = Universe()

at = Atom(uni1)
at.Atom_id = 1
at.globAtom_Index = 0

at.show_info()
at.save("atom.xml")

The instruction above should produce the corresponding `atom.xml` file

Now let's create thee System object and add atoms to it
<a name="System-1"></a><a name="CREATE_ATOM-1"></a>

In [4]:
syst = System()  # chemical system

syst.CREATE_ATOM( Atom(U,  {"Atom_element":"C","Atom_cm_x":0.0,"Atom_cm_y":0.0,"Atom_cm_z":0.0})  )
syst.CREATE_ATOM( Atom(U,  {"Atom_element":"H","Atom_cm_x":-1.0,"Atom_cm_y":-1.0,"Atom_cm_z":0.0}) )

<a name="CREATE_ATOM-2"></a>
We may also add the ATOM objects separately and then add them to the System

In [5]:
at = Atom(U, {"Atom_element":"H","Atom_cm_x":-1.0,"Atom_cm_y":1.0,"Atom_cm_z":0.0}) 
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"C","Atom_cm_x":2.0,"Atom_cm_y":0.0,"Atom_cm_z":0.0})    
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"H","Atom_cm_x":3.0,"Atom_cm_y":1.0,"Atom_cm_z":0.0})   
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"H","Atom_cm_x":3.0,"Atom_cm_y":-1.0,"Atom_cm_z":0.0}) 
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"Au","Atom_cm_x":0.0,"Atom_cm_y":0.0,"Atom_cm_z":-2.0}) 
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"Au","Atom_cm_x":1.0,"Atom_cm_y":0.0,"Atom_cm_z":-2.0}) 
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"Au","Atom_cm_x":0.0,"Atom_cm_y":1.0,"Atom_cm_z":-2.0}) 
syst.CREATE_ATOM(at)

at = Atom(U, {"Atom_element":"Au","Atom_cm_x":1.0,"Atom_cm_y":1.0,"Atom_cm_z":-2.0}) 
syst.CREATE_ATOM(at)


Let's show the atoms belonging to this system. Also, let's show the bonds too (there should be none).

**NOTE**: look in the terminal for the output, nothing will show up in the Jupyter notebook
<a name="show_atoms-1"></a><a name="show_bonds-1"></a><a name="show_molecules-1"></a><a name="show_fragments-1"></a>

In [6]:
print("Showing Atoms")
syst.show_atoms()

print("Showing Bonds")
syst.show_bonds()

print("Showing Molecules")
syst.show_molecules()

print("Showing Fragments")
syst.show_fragments()

Showing Atoms
Showing Bonds
Showing Molecules
Showing Fragments


In the output, you can see that there are:
    
* 10 atoms
* 0 bonds
* 10 fragments - each atom is fragment
* 10 molecules - each atom is also a molecule

### 2.2 Linking and grouping atoms
<a name="2.2."></a>[Back to TOC](#TOC)

We can now link atoms to each other (using atom numbers = starting from 1, not from 0) and the print the information about atoms, bonds, fragments, and molecules again.
<a name="LINK_ATOMS-1"></a>

In [7]:
print("Linking atoms")
syst.LINK_ATOMS(1,2)
syst.LINK_ATOMS(1,3)
syst.LINK_ATOMS(1,4)
syst.LINK_ATOMS(4,5)
syst.LINK_ATOMS(4,6)

print("Showing Bonds")
syst.show_bonds()

print("Showing Molecules")
syst.show_molecules()

print("Showing Fragments")
syst.show_fragments()


Linking atoms
Showing Bonds
Showing Molecules
Showing Fragments


Now, you can see that there are:


* 10 atoms
* 5 bonds - the output shows the indices (starting from 0) of the corresponding atoms, not their numbers (starting from 1)
* 10 fragments - each atom is a separate fragment still
* 5 molecules - the first molecule consists of the first 6 atoms that are linked together and the other 4 molecules are just the 4 remining unlinked (free) atoms

Next, let's group atoms to define fragments. This is done like this:
<a name="GROUP_ATOMS-1"></a>

In [8]:
print("Grouping atoms")

syst.GROUP_ATOMS([1,2,3],1)
syst.GROUP_ATOMS([4,5,6],2)
syst.GROUP_ATOMS([7,8,9,10],3)
syst.show_fragments()


Grouping atoms


Now one can see that there are 3 groups (fragments) in the output.

Note that the indices of the created groups are not congruent with the order of their creation. However, one can see that the group IDs are what one identified at the moment of their creation.

### 2.3 Functional group identification
<a name="2.3."></a>[Back to TOC](#TOC)

Now when we have linked atoms, their types can be determined using the `determine_functional_groups` function.

The parameter of the function means:

* 0 - do not check whether atoms belong to rings;
* 1 - do determine it - this may affect the functional group definition and atom types determined
<a name="determine_functional_groups-1"></a>

In [9]:
syst.determine_functional_groups(0)
syst.show_atoms()

You can see that now H and C atoms belong to the Alkenyl functional group (that is C is sp^3-hybridized).

## 3. Automatic building of a System object using the `LoadMolecule` module
<a name="3."></a>[Back to TOC](#TOC)

### 3.1. Systems without rings
<a name="3.1."></a>[Back to TOC](#TOC)

All the above steps of atom creation, linking and grouping can be done automatically using the `Load_Molecule` function of the `libra_py.LoadMolecule` module. 

As one of the argument, the function takes the expected file format. Here, the `pdb_1` format goes like this:


    HETATM    1  C           0       1.275   0.350   0.000                       C
    HETATM    2  H           0       1.632   0.854   0.874                       H
    HETATM    3  H           0       1.632   0.854  -0.874                       H
    ...
    HETATM   11  F           0       1.723  -3.101   1.256                       F
    END
    CONECT    1    2    3    4    5
    CONECT    2    1
    CONECT    3    1
    ...
    CONECT   10    8
    CONECT   11    8


Atomic coordinates are in **Angstrom**

Here, we are going to construct several molecules from the supplied input files. After loading the molecules (and all the linking a grouping), the functional group will be determined and the atom types will appear in the terminal/console output.
<a name="Load_Molecule-1"></a>

In [10]:
i = 1
while i<=19:
    print("=====================System", i,"=====================")
    # System creation section
    syst = System()

    # Create atoms, link them, define groups
    inp_file = os.getcwd()+"/Molecules/test"+str(i)+"a.pdb"
    LoadMolecule.Load_Molecule(U, syst, inp_file, "pdb_1")

    syst.determine_functional_groups(1)  # 1 - determine rings
    syst.show_atoms()
    print("=======================================================")

    i = i + 1


=====================System 1 =====================
24
=====================System 2 =====================
13
=====================System 3 =====================
35
=====================System 4 =====================
15
=====================System 5 =====================
23
=====================System 6 =====================
17
=====================System 7 =====================
23
=====================System 8 =====================
21
=====================System 9 =====================
21
=====================System 10 =====================
27
=====================System 11 =====================
19
=====================System 12 =====================
23
=====================System 13 =====================
17
=====================System 14 =====================
27
=====================System 15 =====================
29
=====================System 16 =====================
27
=====================System 17 =====================
15
=====================System 18 =====================
21
=

### 3.2. Systems with rings
<a name="3.2."></a>[Back to TOC](#TOC)

Below, we are doing the same as before, but now molecules have stand-alone or fused rings. We now run the `determine_functional_group` function with the argument of `1` to tell the function to determine the minimal set of minimal rings. 

As you can see, run times are okay for most of the systems. It takes a minute or two for the system 12, which is fullerene, although it still works.
<a name="show_rings-1"></a>

In [ ]:
t = Timer()

i = 1
while i<=12:
    print("=====================System", i,"=====================")
    # System creation section
    syst = System()

    # Create atoms, link them, define groups
    inp_file = os.getcwd()+"/Rings/test"+str(i)+".pdb"
    LoadMolecule.Load_Molecule(U, syst, inp_file, "pdb_1")

    t.start()
    syst.determine_functional_groups(1)  # 1 - determine rings
    t.stop()
    print("Time to compute = ", t.show(), " sec")

    syst.show_rings()
    syst.show_atoms()
    print("=======================================================")

    i = i + 1


### 3.3. Loading multiple molecules
<a name="3.3."></a>[Back to TOC](#TOC)

Now, we can load systems with multiple disconnected groups of atoms (molecules). In this case, the format is `pdb`:


    HETATM    1  O        1       0.705   0.744   0.160    -0.20
    HETATM    2  H        1      -0.071   0.264   0.450     0.10
    HETATM    3  H        1       1.356   0.064  -0.014     0.10
    HETATM    4  O        2      -1.092  -0.672   1.792    -0.20
    HETATM    5  H        2      -0.725  -1.146   2.538     0.10
    HETATM    6  H        2      -1.282   0.203   2.132     0.10

    CONECT    1    2    3
    CONECT    2    1
    CONECT    3    1
    CONECT    4    5    6
    CONECT    5    4
    CONECT    6    4

    GROUP    1   2   3       FRAGNAME  WAT  O  H  H
    GROUP    4   5   6       FRAGNAME  WAT  O  H  H


Here, the `GROUP` section defined which atoms are grouped together. "FRAGNAME" is an expected keyword, "WAT" is the  name of the group - doesn't matter too much. The next 3 letters also don't matter too much, but should be there.

In [11]:
# System 1
print("============ Deal with system 1 (2 water molecules) ==============")
syst1 = System()
LoadMolecule.Load_Molecule(U, syst1, os.getcwd()+"/Clusters/2waters.ent", "pdb")
syst1.determine_functional_groups(1)  # 1 - determine rings

syst1.show_atoms()
syst1.show_fragments()
syst1.show_molecules()

# System 2
print("============ Deal with system 2 (2 benzene molecules) ==============")
syst2 = System()
LoadMolecule.Load_Molecule(U, syst2, os.getcwd()+"/Clusters/2benz.ent", "pdb")
syst2.determine_functional_groups(0)  # 

syst2.show_atoms()
syst2.show_fragments()
syst2.show_molecules()



============ Deal with system 1 (2 water molecules) ==============
18
GROUP    1   2   3       FRAGNAME  WAT  O  H  H

GROUP    4   5   6       FRAGNAME  WAT  O  H  H

============ Deal with system 2 (2 benzene molecules) ==============
53
GROUP  1  2  3  4  5  6  7  8  9 10 11 12           FRAGNAME  benz    C C H

GROUP 13 14 15 16 17 18 19 20 21 22 23 24           FRAGNAME  benz    C C H



### 3.4. Constructing periodic Lattices
<a name="3.4."></a>[Back to TOC](#TOC)

The code below, will demonstrate how to construct a periodic slab of Au(111) surface and save it as an .xyz file as well as the .pdb file that also stores the supercell information. Such .pdb files can be loaded into VMD to also demonstrate the periodic images. Remember - the regular .xyz file doesn't have information about supercell vectors, so only the original system (but not its replicas can be shown in the VMD).
<a name="init_box-1"></a><a name="print_ent-1"></a>

In [12]:
# Conversion parameters
Angst_to_Bohr = 1.889725989  

#define constants
a = 4.07825
Nx = 6
Ny = 6
Nz = 2

#define lattice vector (unit cell)
t1 = VECTOR(0.5*a*math.sqrt(2.0),        0.0,                         0.0 )
t2 = VECTOR(0.5*a*math.sqrt(1.0/2.0),    0.5*a*math.sqrt(3.0/2.0),    0.0 )
t3 = VECTOR(0.0,                         0.0,                         a*math.sqrt(3.0) )

# and the transformation matrix
O = MATRIX(3,3)
O.set(0,0,t1.x); O.set(0,1,t2.x); O.set(0,2,t3.x)
O.set(1,0,t1.y); O.set(1,1,t2.y); O.set(1,2,t3.y)
O.set(2,0,t1.z); O.set(2,1,t2.z); O.set(2,2,t3.z)

#define atomic basis
Au = []
Au.append( VECTOR( 0.0,      0.0,       0.0   ) )
Au.append( VECTOR(1.0/3.0,  1.0/3.0,   1.0/3.0) )
Au.append( VECTOR(2.0/3.0,  2.0/3.0,   2.0/3.0) )



#Compute the number of atoms,using input information: Nx, Ny, Nz
Nat = len(Au)*Nx*Ny*Nz   

# Create a Universe and the empty system
U = Universe(); LoadPT.Load_PT(U, os.getcwd()+"/elements.dat")
syst = System()


#compute atomic coordinates and print them into the file
myFile = open('atoms.xyz','w')
myFile.write('%5i\n' % Nat)
myFile.write('\n')

for nx in range(Nx):
    for ny in range(Ny):
        for nz in range(Nz):

            for i in range(0,3):
                # Crystal coordinates = unit cell atomic basis + integer translations
                au_i_cryst = VECTOR( Au[i].x + nx, Au[i].y + ny, Au[i].z + nz )

                #transform crystal coordinates to Cartesian
                au_i_cart = O * au_i_cryst
            
                # Print coordinates
                myFile.write('Au\t% 16.5f\t% 16.5f\t% 16.5f\n' % (au_i_cart.x, au_i_cart.y, au_i_cart.z))  # in Angstrom

                # Create Au atoms and add them to the chemical system
                at = Atom(U, {"Atom_element": "Au", 
                              "Atom_cm_x": au_i_cart.x * Angst_to_Bohr,
                              "Atom_cm_y": au_i_cart.y * Angst_to_Bohr, 
                              "Atom_cm_z": au_i_cart.z * Angst_to_Bohr
                             } )
                syst.CREATE_ATOM(at)  # internally, we assume Bohrs, so the conversion is made up there

#close the output file
myFile.close()

# Compute the supercell translation vectors
T1 = Nx*t1; T2 = Ny*t2; T3 = Nz*t3

# Convert lattice vectors to Bohrs
T1 *= Angst_to_Bohr
T2 *= Angst_to_Bohr
T3 *= Angst_to_Bohr

print("Lattice vectors")
print("T1 = ", T1.x, T1.y, T1.z)
print("T2 = ", T2.x, T2.y, T2.z)
print("T3 = ", T3.x, T3.y, T3.z)

syst.init_box(T1,T2,T3)
syst.print_ent("au.pdb")


Lattice vectors
T1 =  32.697077243582804 0.0 0.0
T2 =  16.348538621791402 28.316499522444772 0.0
T3 =  0.0 0.0 26.697051775715114


## 4. Manipulating System objects
<a name="4."></a>[Back to TOC](#TOC)

### 4.1. Rotating fragments
<a name="4.1."></a>[Back to TOC](#TOC)

Once the System object is constructed, one can transform molecules, fragments, and atoms of the system by translating and rotating (excluding atoms) them. The rotation is performed with the help of one of the following functions:
<a name="ROTATE_FRAGMENT-1"></a>

    void ROTATE_FRAGMENT(double degree_amount, const VECTOR& direction_lab, int fr_id, const VECTOR& center);
    void ROTATE_FRAGMENT(double degree_amount, const VECTOR& direction_lab, int fr_id, int center_indx);
    void ROTATE_FRAGMENT(double degree_amount, const VECTOR& direction_lab, int fr_id);
    void ROTATE_FRAGMENT(const MATRIX3x3& rot, int fr_id, const VECTOR& center);
    void ROTATE_FRAGMENT(const MATRIX3x3& rot, int fr_id, int center_indx);
    void ROTATE_FRAGMENT(const MATRIX3x3& rot, int fr_id);
    
Here,

* `degree_amount` is the amount of rotation in degrees 
* `direction_lab` is the direction of rotation (axis) in the laboratory coordinate system (external)
* `fr_id` is the ID of the fragment we want to rotate
* `center` specifies the pivot point, if not specified the pivot point is the same as the fragment's center of mass
* `center_indx` is the index of the atom that is taken to be the pivot center.


Note that the function changes the system to which it is applied so **the rotation operation has the cumulative effect**

Below, I demonstate how to rotate a fragment around a certain pivot point in a given direction. Specifically, we will look at the C2H4 molecule and will rotate one CH2 group along the C=C double bond.


Let's create a molecular system by loading it from the file. The file also defines the fragments via the `Load_Molecule` function. However, we still need to compute the properties of all the fragments by using the `init_fragments` function. This function will compute things like fragment mass, moment of inertia, center of mass, and the orientation matrix.

We can also print out the statistics of our system such as the number of atoms, bonds, etc.
<a name="init_fragments-1"></a><a name="Number_of_atoms-1"></a><a name="Number_of_bonds-1"></a><a name="Number_of_angles-1"></a><a name="Number_of_dihedrals-1"></a><a name="Number_of_impropers-1"></a><a name="Number_of_fragments-1"></a><a name="Atoms-1"></a><a name="Atom_RB-1"></a>

In [13]:
# Create molecular system and initialize the properties                                                           
syst0 = System();  LoadMolecule.Load_Molecule(U, syst0, os.getcwd()+"/Molecules/c2h4.ent", "pdb")                                                                                                                
syst0.determine_functional_groups(0)  # do not assign rings
syst0.init_fragments()

print("Number of atoms in the system = ", syst0.Number_of_atoms)
print("Number of bonds in the system = ", syst0.Number_of_bonds)
print("Number of angles in the system = ", syst0.Number_of_angles)
print("Number of dihedrals in the system = ", syst0.Number_of_dihedrals)
print("Number of impropers in the system = ", syst0.Number_of_impropers)
print("Number of fragments in the system = ", syst0.Number_of_fragments)
                                                                                                                  
                                                                                                                  
# Generate configurations
print("Printing coordinates of all atoms")
for i in range(syst0.Number_of_atoms):
    x = syst0.Atoms[i].Atom_RB.rb_cm.x
    y = syst0.Atoms[i].Atom_RB.rb_cm.y
    z = syst0.Atoms[i].Atom_RB.rb_cm.z
    print(i, x, y, z)
    
syst0.show_atoms()


19
GROUP    1   2   3       FRAGNAME  WAT  O  H  H

GROUP    4   5   6       FRAGNAME  WAT  O  H  H

Number of atoms in the system =  6
Number of bonds in the system =  5
Number of angles in the system =  6
Number of dihedrals in the system =  4
Number of impropers in the system =  2
Number of fragments in the system =  2
Printing coordinates of all atoms
0 0.133340955509829 1.8066574139755378 0.0
1 1.04673812256699 -0.09259468373501101 0.0
2 -1.9680456826141048 1.647111628176246 0.0
3 1.372636487177952 3.9540891155474465 0.0
4 0.45923932012079105 5.853341213257995 0.1889725989
5 3.474023125301886 4.113634901346738 0.1889725989


Now, we are ready to rotate a fragment. This can be done via `ROTATE_FRAGMENT` function, which comes in several signatures. In particular, here we can use (comment/uncomment) the one which defines the pivot center explicitly or  the one that selects one of the atoms of the system to be the pivot center.

One way to check that the `ROTATE_FRAGMENT` function works correctly is to show that the point that belongs to the rotation axis doesn't move. This is checked by printing line at the bottom.

Note that we can select either atom 0 or 3 as the pivot center - in both cases, the atom 0 belonging to the rotated fragment should not move.
<a name="ROTATE_FRAGMENT-2"></a><a name="Fragments-1"></a><a name="print_xyz-1"></a>

In [14]:
fr_num = 1     # ID of the fragment we are going to rotate
atom_indx = 0  # index of the atom that serves as a pivot center
pt = VECTOR(syst0.Atoms[0].Atom_RB.rb_cm)
u = VECTOR(syst0.Atoms[0].Atom_RB.rb_cm - syst0.Atoms[3].Atom_RB.rb_cm)  # C-C direction
print("The rotation axis is: ", u.x, u.y, u.z)
print("The rotation center is: ", pt.x, pt.y, pt.z)

f = open("c2h4_0.xyz","w");  f.close()
for i in range(180):
    # One way
    #syst0.ROTATE_FRAGMENT(2.0, u, fr_num, pt)
    
    # Another way
    syst0.ROTATE_FRAGMENT(2.0, u, fr_num, atom_indx)
            
    syst0.print_xyz("c2h4_0.xyz",i)
        
    center = syst0.Atoms[0].Atom_RB.rb_cm
    #center = syst0.Fragments[0].Group_RB.rb_cm
    x,y,z = center.x, center.y, center.z
    Lt = syst0.Fragments[0].Group_RB.rb_L
    
    print(F"i = {i}  x = {x}  y = {y}  z = {z}" )     

The rotation axis is:  -1.239295531668123 -2.1474317015719087 0.0
The rotation center is:  0.133340955509829 1.8066574139755378 0.0
i = 0  x = 0.133340955509829  y = 1.8066574139755378  z = -7.607414657527685e-19
i = 1  x = 0.133340955509829  y = 1.8066574139755378  z = -1.4466263947919382e-18
i = 2  x = 0.133340955509829  y = 1.8066574139755376  z = -1.6135977645194421e-18
i = 3  x = 0.133340955509829  y = 1.8066574139755376  z = -3.6801463975831214e-18
i = 4  x = 0.13334095550982897  y = 1.8066574139755376  z = -5.1821475713018095e-18
i = 5  x = 0.13334095550982897  y = 1.8066574139755376  z = -6.1414123840673046e-18
i = 6  x = 0.13334095550982897  y = 1.8066574139755378  z = -9.21825956496855e-18
i = 7  x = 0.13334095550982897  y = 1.8066574139755378  z = -9.427053186466736e-18
i = 8  x = 0.13334095550982894  y = 1.8066574139755378  z = -1.1647973574167511e-17
i = 9  x = 0.13334095550982894  y = 1.806657413975538  z = -1.4288598787232792e-17
i = 10  x = 0.13334095550982894  y = 1.80

i = 159  x = 0.13334095550982866  y = 1.8066574139755376  z = -1.6737362567415502e-15
i = 160  x = 0.13334095550982866  y = 1.8066574139755376  z = -1.6929554343147503e-15
i = 161  x = 0.13334095550982866  y = 1.8066574139755378  z = -1.7113157204794345e-15
i = 162  x = 0.13334095550982866  y = 1.8066574139755378  z = -1.7274813442777828e-15
i = 163  x = 0.13334095550982866  y = 1.8066574139755378  z = -1.744349158389405e-15
i = 164  x = 0.13334095550982866  y = 1.8066574139755378  z = -1.7624231474179172e-15
i = 165  x = 0.13334095550982866  y = 1.8066574139755378  z = -1.7820370423445378e-15
i = 166  x = 0.13334095550982863  y = 1.806657413975538  z = -1.8022150612140297e-15
i = 167  x = 0.13334095550982863  y = 1.8066574139755378  z = -1.81975287838693e-15
i = 168  x = 0.1333409555098286  y = 1.806657413975538  z = -1.8404073532892525e-15
i = 169  x = 0.1333409555098286  y = 1.8066574139755378  z = -1.8562710098419047e-15
i = 170  x = 0.1333409555098286  y = 1.8066574139755378  z = 

The trajectory showing the CH2 rotation along the C=C double bond is stored in the "c2h4_0.xyz" file and can be visualized using VMD or py3Dmol snippet below.

In [ ]:
xyz = ""
f = open("c2h4_0.xyz","r")
for a in f.readlines():
    xyz = xyz + a
f.close()

view = py3Dmol.view(width=800,height=400)  
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(xyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'forward', 'step':1})
view.show()  


### 4.2. Translating fragments
<a name="4.2."></a>[Back to TOC](#TOC)

Analogously to the rotation, we can translate fragments and atoms of the chemical system using the `TRANSLATE_ATOM` or `TRANSLATE_FRAGMENT` functions:
<a name="TRANSLATE_FRAGMENT-1"></a>

    void TRANSLATE_ATOM(double amount, VECTOR dir, int fr_id);
    void TRANSLATE_FRAGMENT(double amount, VECTOR dir, int fr_id);
  
The arguments of the function are:

* `amount` the amount of translation (in whatever units are used, a.u. by default)
* `dir` is the direction of translation - the magnitude doesn't matter, in the lab coordinates
* `fr_id` is the ID of the fragment to translate

The translation example is illustrated below

In [15]:
syst1 = System();  LoadMolecule.Load_Molecule(U, syst1, os.getcwd()+"/Molecules/c2h4.ent", "pdb")                                                                                                                
syst1.determine_functional_groups(0)  # do not assign rings
syst1.init_fragments()

fr_num = 1     # ID of the fragment we are going to rotate

u = VECTOR(syst1.Atoms[0].Atom_RB.rb_cm - syst1.Atoms[3].Atom_RB.rb_cm)  # C-C direction
print("The translation axis is: ", u.x, u.y, u.z)

f = open("c2h4_1.xyz","w");  f.close()
for i in range(50):
    # One way
    syst1.TRANSLATE_FRAGMENT(0.1, u, fr_num)        
    syst1.print_xyz("c2h4_1.xyz",i)


19
GROUP    1   2   3       FRAGNAME  WAT  O  H  H

GROUP    4   5   6       FRAGNAME  WAT  O  H  H

The translation axis is:  -1.239295531668123 -2.1474317015719087 0.0


The trajectory can be visualized with the snippet below:

In [ ]:
xyz = ""
f = open("c2h4_1.xyz","r")
for a in f.readlines():
    xyz = xyz + a
f.close()

view = py3Dmol.view(width=800,height=400)  
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(xyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'forward', 'step':1})
view.show()  